In [17]:
import requests, json
import psycopg2


In [ ]:
# Extract:
acc_url = "https://apis.haravan.com/com/orders.json"

acc_info = json.loads(requests.get(acc_url).text)

print(acc_info)

In [13]:
# Transform:
def transform_data(data):
    transformed_data = []

    # Checks if data is a string
    if isinstance(data, dict) and 'orders' in data:
        orders = data['orders']
        for order in orders:
            # Make the necessary transformers for each order
            transformed_order = {
                'order_customer': order['customer'],
                'customer_name': order['customer']['first_name'],
                'total_price': order['total_price'],
                # Add other fields as required
            }
            transformed_data.append(transformed_order)
    else:
        print("Dữ liệu không hợp lệ. Kiểm tra lại định dạng của data.")

    return transformed_data

# Biến đổi dữ liệu
transformed_orders = transform_data(data)


Dữ liệu không hợp lệ. Kiểm tra lại định dạng của data.


In [ ]:
def transform_data(data):
    transformed_data = {
        'order_customer_id': data['order']['customer']['id'],
        'customer_email': data['order']['email'],
        'shipping_address': {
            'address1': data['order']['shipping_address']['address1'],
            'company': data['order']['shipping_address']['company'],
            'country': data['order']['shipping_address']['country'],
            'first_name': data['order']['shipping_address']['first_name'],
            'last_name': data['order']['shipping_address']['last_name'],
            'phone': data['order']['shipping_address']['phone'],
            'province': data['order']['shipping_address']['province'],
            'district': data['order']['shipping_address']['district'],
            'ward': data['order']['shipping_address']['ward']
        },
        'line_items': [],
        'tags': data['order']['tags'],
        'gateway': data['order']['gateway'],
        'note': data['order']['note'],
        'discount_codes': [],
        'total_discounts': data['order']['total_discounts'],
        'shipping_lines': []
    }

    # Transform data for each product in the order
    for line_item in data['order']['line_items']:
        transformed_item = {
            'variant_id': line_item['variant_id'],
            'quantity': line_item['quantity'],
            'price': line_item['price'],
            'applied_discounts': line_item['applied_discounts'],
            'total_discount': line_item['total_discount'],
            'properties': line_item['properties']
        }
        transformed_data['line_items'].append(transformed_item)

    # Transform data for discount codes
    for discount_code in data['order']['discount_codes']:
        transformed_discount = {
            'amount': discount_code['amount'],
            'code': discount_code['code'],
            'is_coupon_code': discount_code['is_coupon_code']
        }
        transformed_data['discount_codes'].append(transformed_discount)

    # Transform data for shipping lines
    for shipping_line in data['order']['shipping_lines']:
        transformed_shipping_line = {
            'code': shipping_line['code'],
            'price': shipping_line['price'],
            'title': shipping_line['title']
        }
        transformed_data['shipping_lines'].append(transformed_shipping_line)

    return transformed_data

# Sử dụng hàm transform_data
transformed_data = transform_data(data)
print(transformed_data)


In [ ]:

def load_data(data):
    # Kết nối đến kho dữ liệu (PostgreSQL)
    conn = psycopg2.connect("host=localhost dbname=order user=student password=student")
    cur = conn.cursor()

    try:
        for order in transformed_data:
            # Perform an INSERT query into the database
            insert_order_query = """
            INSERT INTO orders (order_customer_id, customer_email, total_price)
            VALUES (%s, %s, %s)
            RETURNING order_id;
            """
            cur.execute(insert_order_query, (order['order_customer_id'], order['customer_email'], order['total_price']))
            order_id = cur.fetchone()[0]

            # Load data for products in the order
            for line_item in order['line_items']:
                insert_line_item_query = """
                INSERT INTO line_items (order_id, product_id, variant_id, quantity, price, total_discount)
                VALUES (%s, %s, %s, %s, %s, %s);
                """
                cur.execute(insert_line_item_query, (order_id, line_item['product_id'], line_item['variant_id'],
                                                     line_item['quantity'], line_item['price'], line_item['total_discount']))

            # Load data for discount codes
            for discount_code in order['discount_codes']:
                insert_discount_query = """
                INSERT INTO discount_codes (order_id, amount, code, is_coupon_code)
                VALUES (%s, %s, %s, %s);
                """
                cur.execute(insert_discount_query, (order_id, discount_code['amount'], discount_code['code'],
                                                    discount_code['is_coupon_code']))

            # Load data for shipping lines
            for shipping_line in order['shipping_lines']:
                insert_shipping_line_query = """
                INSERT INTO shipping_lines (order_id, code, price, title)
                VALUES (%s, %s, %s, %s);
                """
                cur.execute(insert_shipping_line_query, (order_id, shipping_line['code'], shipping_line['price'],
                                                         shipping_line['title']))

        # Commit and close the connection
        conn.commit()
    except Exception as e:
        # Error handling and logging
        print(f"Lỗi khi nạp dữ liệu vào kho dữ liệu tập trung: {e}")
    finally:
        cur.close()
        conn.close()


# Load data into centralized data warehouse
load_data(transformed_orders)


Sample data

In [7]:
data = {
  "order": {
    "customer": {
      "id": 1085558958
    },
    "email": "example@haravan.com",
    "shipping_address": {
      "address1": "182 Lê Đại Hành",
      "company": "HRV",
      "country": "Vietnam",
      "first_name": "A",
      "last_name": "Nguyễn Văn",
      "phone": "0909090909",
      "province": "Hồ Chí Minh",
      "province_code": "HC",
      "country_code": "VN",
      "district_code": "HC476",
      "district": "Quận 11",
      "ward_code": "27208",
      "ward": "Phường 15"
    },
    "send_receipt": False,
    "send_fulfillment_receipt": False,
    "is_confirm": True,
    "source": "pos",
    "source_name": "pos",
    "line_items": [
      {
        "product_id": 1003148078,
        "variant_id": 1008493211,
        "quantity": 1,
        "price": 110000,
        "applied_discounts": [
          {
            "description": "Khuyến mãi sinh nhật",
            "amount": 5000.0
          }
        ],
        "total_discount": 5000.0,
        "properties": [
          {
            "name": "Ghi chú sản phẩm",
            "value": "Xuất xứ Vietnam"
          }
        ]
      },
      {
        "price": 170000.0,
        "quantity": 2,
        "title": "Sản phầm ABC",
        "sku": "ABC",
        "barcode": "ABCDE",
        "applied_discounts": [
          {
            "description": "Khuyến mãi sinh nhật",
            "amount": 30000.0
          }
        ],
        "total_discount": 30000.0,
        "properties": [
          {
            "name": "Ghi chú sản phẩm",
            "value": "Xuất xứ Hà Nội"
          }
        ]
      }
    ],
    "tags": "A1,A2",
    "gateway": "Tiền mặt",
    "note": "Ghi chú đơn hàng",
    "discount_codes": [
      {
        "amount": 10000.0,
        "code": "Khuyến mãi mùa hè",
        "is_coupon_code": False
      }
    ],
    "total_discounts": 10000,
    "shipping_lines": [
      {
        "code": "Giao hàng tận nơi",
        "price": 15000.0,
        "title": "Giao hàng tận nơi"
      }
    ],
    "landing_site_ref": "refabc",
    "location_id": 895636,
    "user_id": "1000374246",
    "note_attributes": [
      {
        "name": "Ghi chú khác",
        "value": "thông tin khác"
      }
    ]
  }
}